In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import nip

import os
import sys
sys.path.append("/home/msst/repo/Quantization")
import qlib

DEVICE = 'cuda:0'

In [2]:
# import torch

# bit_width = 8
# N = -2 ** (bit_width - 1)
# P = 2 ** (bit_width - 1) - 1

# x = torch.randn(3, 2048, 4096)

# reduce_dims = tuple(range(x.dim() - 1))  # (0, 1)
# x_scale = torch.maximum(
# 	x.amin(dim=reduce_dims) / N, 
# 	x.amax(dim=reduce_dims) / P, 
# )

# print(x_scale)

In [3]:
# path_to_model = '/home/msst/repo/Quantization/ml/llm/pretrained_models/Llama2-7b-trellis'
# qmodel = AutoModelForCausalLM.from_pretrained(
# 	path_to_model, 
# 	trust_remote_code=True,
#     torch_dtype=torch.float16,
# ).to(DEVICE)

# tokenizer = AutoTokenizer.from_pretrained(path_to_model)

In [4]:
path_to_checkpoints = "/home/msst/repo/Quantization/logs/checkpoints_Llama2-7b-hf/trellis/"
chpnt_name = 'T256_L16_V2_K2_cbs10_LowBitSym_qtip_ptq_bs5_quant_act'

qmodel = qlib.QuantizedLlamaForCausalLM.from_pretrained(
	os.path.join(path_to_checkpoints, chpnt_name),
    torch_dtype=torch.float16,
).to(DEVICE)

model_name = 'Llama2-7b-hf'
tokenizer = qlib.load_tokenizer(model_name)

LOL, Im custom!


In [5]:
dataloader = qlib.QATDataset(
    config=nip.load('/home/msst/repo/Quantization/configs/data/wikitext_test_seqlen4096.yaml'),
    tokenizer=tokenizer
).get_dataloader()

with torch.amp.autocast('cuda', dtype=torch.float16):
    res = qlib.evaluate(qmodel.half(), dataloader, print_times=25)
    print(res)


 10%|▉         | 2/21 [00:32<05:01, 15.88s/it]

9.761570994929407


 14%|█▍        | 3/21 [00:46<04:27, 14.88s/it]

9.922030666172638


 19%|█▉        | 4/21 [01:00<04:04, 14.41s/it]

...

9.97668398083752


 95%|█████████▌| 20/21 [04:39<00:13, 13.74s/it]

9.94001127813132


100%|██████████| 21/21 [04:50<00:00, 13.83s/it]

10.024664093302395
10.024664093302395


In [6]:
# path_to_save = os.path.join(
# 	path_to_checkpoints, 
# 	f'{chpnt_name}_act8bit'
# )
# qmodel.half().save_pretrained(path_to_save)

In [7]:
### 25.07 ppl init
## 7.016 ptq (ns=128, bs=2, ne=1)
## 7.545 ptq (ns=128, bs=2, ne=1, fp acts)

## 7.058 ptq (ns=128, bs=2, ne=2)
# 6.654 ptq + qat (KD+CE, 25 steps)
# 6.598 ptq + qat (CE, 25 steps)
# 6.547 ptq + qat (KD, 10 steps)
# 6.511 ptq + qat (KD, 5 steps)

## 6.797 ptq (ns=320, bs=5, ne=4)
## 6.779 ptq (ns=320, bs=5, ne=2)
# 6.579 ptq + qat (KD, 2 steps 1 ga Adam)
# 6.493 ptq + qat (KD, 5 steps 1 ga Adam) # PiQA 0.7584 # WINO 0.6606 # ArcE 0.7071 # ArcC 0.3712
# 7.372 ptq + qat (KD, 25 steps 1 ga Adam)
# 7.236 ptq + qat (KD, 10 steps 4 ga Adafactor)
# > 7.5 ptq + qat (CE, 10 steps 4 ga Adafactor)
# 6.567 ptq + qat (CE, 10 steps 4 ga Adam)
# 6.612 ptq + qat (CE, 10 steps 4 ga Adam small lr)
# 6.609 ptq + qat (CE, 25 steps 4 ga Adam)
# 7.449 ptq + qat (CE, 100 steps 2 ga Adam)
# 6.464 ptq + qat (CE, 20 steps 10 ga Adam)
# 6.390 ptq + qat (CE, 20 steps 20 ga Adam)

### 39 ppl init
# 6.778 ptq (ns=128, bs=2, ne=2)
# 6.565 ptq + qat (CE, 10 steps 1 ga Adafactor)
# 6.478 ptq + qat (KD, 25 steps 1 ga Adafactor)
# 6.420 ptq + qat (KD+CE, 25 steps 1 ga Adafactor)